In [ ]:
import itertools 
import random
def train_model(Q_copy, env_inv, num_episodes = 10000,  epsilon_final= 0.1, epsilon = 0.8, epsilon_decay = 0.999,  learning_rate = 0.9,
            gamma = 0.90 ,   to_print = True):
    steps_total_inv = [] # store number of steps taken in each episode
    rewards_total_inv = [] #store reward obtained for each episode
    epsilon_total_inv = [] #store epsilon obtained at the end of each episode
    terminal_state_inv = []
    if len(Q_copy) == 0:
        Q_copy = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
        Q_copy.loc[15] = np.zeros(n_actions,)
    for i_episode in range(num_episodes):

        # resets the environment
        state = env_inv.reset()
        step = 0
        #reward = 0

      ## as epsilon decays with more timesteps, the prob. of selecting a random val < e decays --> more likely to exploit. 
        if epsilon > epsilon_final:
                epsilon *= epsilon_decay

        while True:

            step += 1

            random_for_epsilon = np.random.rand()
            if random_for_epsilon <= epsilon:
              action = env_inv.action_space.sample()
            else: 
              action = np.argmax(Q_copy.loc[state])


            ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
            new_state, reward , done, info = env_inv.step(action)

            ##if you want reward penalized at for each timestep
            #reward = rewarder(new_state, reward)

            # filling the Q Table - 

            Q_copy.loc[state, action] = (1- learning_rate)*Q_copy.at[state, action] + learning_rate*(reward + gamma * np.max(Q_copy.loc[new_state]))

            # Setting new state for next action
            state = new_state
            tile = og_4x4_inv[rowsandcols(state)[0]][rowsandcols(state)[1]]
            #env.render()

            if done:
              #print(Q)

              terminal_state_inv.append(tile)
              steps_total_inv.append(step)
              rewards_total_inv.append(reward)
              #print(reward)
              epsilon_total_inv.append(epsilon)
              if to_print:
                  if i_episode % 10 == 0:
                    print('Episode: {} Reward: {} Steps Taken: {} Terminal State: {}, Epsilon: {}'.format(i_episode,reward, step, tile, epsilon))
              if i_episode == num_episodes:
                print('Mean Award %g'%np.mean(rewards_total_inv))
            
              break
    return terminal_state_inv, steps_total_inv, rewards_total_inv, epsilon_total_inv, Q_copy


In [ ]:
def restrict_actions(Q, n_states, n_rows):

  Q.at[n_states -1, :] = np.zeros(n_rows,)
  for i in range( 0, n_states, n_rows): 
    Q.at[i,0] = np.NaN
  for i in range( n_rows -1 , n_states, n_rows): 
    Q.at[i,2] = np.NaN
  for i in range(0, n_rows):
    Q.at[i,3] = np.NaN
  for i in range(n_states - n_rows , n_states):
    Q.at[i,1 ]= np.NaN
  
  return Q
  

In [ ]:
def choose_action(Q, state, epsilon):
  random_for_epsilon = np.random.rand()
  if random_for_epsilon <= epsilon:
    s = Q.loc[state].notna()
    vals = s[s].index.values
    action = random.choice(vals)
  else: 
    Q.loc[state] += np.random.rand(n_actions,)/100
    action = np.argmax(Q.loc[state])
  return action

In [ ]:
def rowsandcols(state):
  ''' input: state returned by env
      output: location of state as (row,col) tuple'''
  return int(np.where(state_matrix ==state)[0]), int(np.where(state_matrix ==state)[1])

In [ ]:
def rewarder(new_state, reward):
  if og_4x4[rowsandcols(new_state)[0]][rowsandcols(new_state)[1]]== 'H':
    reward -= 20
  elif og_4x4[rowsandcols(new_state)[0]][rowsandcols(new_state)[1]]== 'F':
    reward -= 1 
  elif og_4x4[rowsandcols(new_state)[0]][rowsandcols(new_state)[1]]== 'S':
    reward -= 1
  else: #goal
    reward += 100

  return reward

In [ ]:
def check_transfer(env,env2, num_episodes = 10000,  epsilon_final= 0.1, epsilon = 0.8, epsilon_decay = 0.999,  learning_rate = 0.9,
                   gamma = 0.90 ,num_episodes2 =  10000 , epsilon2 = 0.8,r_seed = 90,to_print = False):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    np.random.seed(r_seed)
    Q = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
    Q.loc[15] = np.zeros(n_actions,)
    terminal_state, steps_total, rewards_total, epsilon_total_inv, Q = train_model(Q , env, num_episodes,  epsilon_final, epsilon , epsilon_decay ,  learning_rate,  gamma,to_print = to_print  )        
    Q_copy = Q.copy()
    terminal_state_inv, steps_total_inv, rewards_total_inv, epsilon_total_inv, Q_new = train_model(Q_copy, env2, num_episodes2,  epsilon_final, epsilon2 , epsilon_decay ,  learning_rate,  gamma  ,  to_print = to_print)   
    return Q, Q_new, rewards_total,steps_total,  rewards_total_inv,steps_total_inv


In [ ]:
def movmean(sig, wind = 10):
    av_sig = [np.mean(sig[i-int(wind/2) : i+int(wind/2)]) for i in np.arange(len(sig)-wind)+int(wind/2)+1]
    return av_sig

In [ ]:
def apply_env_transform(board, transform_type = 'transpose',params = {'miss_num':2}, unique_board = np.array(['F','H'])):
    if transform_type == 'transpose':
         tra_board = [''.join([word[i] for word in og_4x4])  for i in range(len(board[0]))]
    elif transform_type == 'miss':
        #if isinstance(board,list): board = np.vstack(board)
        board_new = np.vstack([[letter for letter in row] for row in  board]  )
        pairs = list(itertools.product(np.arange(board_new.shape[0]), np.arange(board_new.shape[1])))
        pairs = pairs[1:-1]
        sample_random = random.sample(pairs,params['miss_num'])
        if len(unique_board) == 0: unique_board = np.unique(board)
        tra_board = board_new.copy()
        #print(sample_random)
        #display(board_new)
        for pair_choose in sample_random:
            #print(pair_choose)
            #print(pair_choose)
            
            cur_value = board_new[pair_choose[0],pair_choose[1]][0]

            new_val = random.choice(unique_board[unique_board != cur_value])

            tra_board[pair_choose[0],pair_choose[1]] = new_val[0]
    else:
        raise NameError('Unknown transform type')
    return tra_board

In [1]:
def from_array_to_list(board_array):
    return [''.join(row) for row in board_array]

In [ ]:
#pairs = list(itertools.combinations(np.arange(num_subdyns),k))

In [ ]:
def map_to_nums(board,dict_trans = {'F' : 1, 'G' : 2, 'H': 3, 'S' : 4}):
    if isinstance(board,list):
        board = np.vstack([[letter for letter in row] for row in  board]  )
        #board = np.vstack(board)
    board_num = np.nan*np.zeros(np.array(board).shape)
    #print(board_num)
    for row_num, word in enumerate(board):
        for col_num, letter in enumerate(word):
            new_val = dict_trans[letter]
            board_num[row_num,col_num] = new_val
    return board_num
                                    
            
        